In [3]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
import matplotlib.pyplot as plt
from Dense import Dense
from Convolutional import Convolutional
from Reshape import Reshape
from Activations import ReLu, Sigmoid, Softmax
from Losses import binary_cross_entropy, binary_cross_entropy_prime
from Network import train, predict
from Layer import Layer

def preprocess_data(x, y, limit):
    zero_index = np.where(y == 0)[0][:limit]
    one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack((zero_index, one_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = np_utils.to_categorical(y)
    y = y.reshape(len(y), 2, 1)
    return x, y

def preprocess_data_all_digits(x, y, limit):
    all_indices = np.random.permutation(len(x))[:limit]
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = np_utils.to_categorical(y)
    y = y.reshape(len(y), 10, 1)
    return x, y

# load MNIST from server, limit to 100 images per class since we're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 60000)
x_test, y_test = preprocess_data(x_test, y_test, 1000)

network = [
    Convolutional((1, 28, 28), 3, 10),
    Sigmoid(),
    Reshape((10, 26, 26), (10 * 26 * 26, 1)),
    Dense(10 * 26 * 26, 100),
    Sigmoid(),
    Dense(100, 2),
    Sigmoid()
]




In [4]:
# train
train(
    network,
    binary_cross_entropy,
    binary_cross_entropy_prime,
    x_train,
    y_train,
    epochs=10,
    learning_rate=0.01,
    batch_size=1
)

/Users/mattia/Desktop/Neural_Nets/Activations.py:75: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


In [ ]:
# test, keep track of correct predictions
correct = 0
for x, y in zip(x_test, y_test):
    output = predict(network, x)
    if np.argmax(output) == np.argmax(y):
        correct += 1
    else:
        #Plot the image
        plt.imshow(x.reshape(28, 28))
        plt.show()
        print(output)

# print the accuracy
print(f"{correct / len(x_test) * 100}% of test data correctly predicted")


In [ ]:
#Now, we create another network using tensorflow and all the digits
x_train, y_train = preprocess_data_all_digits(x_train, y_train, 60000)
x_test, y_test = preprocess_data_all_digits(x_test, y_test, 1000)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(3, kernel_size=(3, 3), activation="sigmoid"),
        layers.Flatten(),
        layers.Dense(100, activation="sigmoid"),
        layers.Dense(10, activation="sigmoid"),
    ]
)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=1, epochs=10, validation_split=0.1)

model.evaluate(x_test, y_test, batch_size=1)

# test, keep track of correct predictions
correct = 0
for x, y in zip(x_test, y_test):
    output = model.predict(x.reshape(1, 28, 28, 1))
    if np.argmax(output) == np.argmax(y):
        correct += 1
    else:
        #Plot the image
        #plt.imshow(x.reshape(28, 28))
        #plt.show()
        #print(output)
        pass